# Data Ingestion from BigQuery

<strong>Step 1: Exploratory Data Analysis</strong>

In [ ]:
%%bigquery all_artists_df

SELECT artist_name,
       count(*) AS num_listens_per_artist
FROM   `listenbrainz.listenbrainz.listen`
GROUP  BY artist_name
ORDER  BY count(*) DESC; 

In [ ]:
all_artists_df.head()

In [ ]:
all_artists_df.shape

In [ ]:
all_artists_df.num_listens_per_artist.plot(kind='hist')

In [ ]:
all_artists_df.describe()

In [ ]:
all_artists_df[all_artists_df.num_listens_per_artist > 58000].shape

In [ ]:
359/1098460

In [ ]:
all_artists_df[all_artists_df.num_listens_per_artist > 50000].plot(kind='hist')

<strong>Step 2: Ingest Artists Data from Listen Brainz Dataset</strong>

This query gets a list of the top 300 most popular artists from the Listen Brainz data table, as determined by the number of user listen events.

In [ ]:
%%bigquery artists_df

SELECT artist_name
FROM   `listenbrainz.listenbrainz.listen`
GROUP  BY artist_name
ORDER  BY count(*) DESC
LIMIT  300; 

In [ ]:
artists_df.head(10).sort_values(by='artist_name')

<strong>Step 3: Ingest User Listening Data</strong>

This query gets a corresponding list of user listens for each artist in the top 300 list.

In [ ]:
%%bigquery play_counts_df

SELECT user_name,
       artist_name,
       count(*) AS cnt
FROM   `listenbrainz.listenbrainz.listen`
WHERE  artist_name IN (SELECT artist_name
                       FROM   `listenbrainz.listenbrainz.listen`
                       GROUP  BY artist_name
                       ORDER  BY count(*) DESC
                       LIMIT  300)
GROUP  BY user_name,
          artist_name; 

In [ ]:
play_counts_df.head()

In [ ]:
play_counts_df.isna().sum()

In [ ]:
play_counts_df.shape

<strong>Step 3: Write the two ingested datasets out to static files in the working directory</strong>

The output files are in the 'feather' format, which is a portable file format for storing pandas dataframes. See [here](https://arrow.apache.org/docs/python/feather.html) for more details.

In [ ]:
artists_df.to_feather("input_data/artist_df.feather")

In [ ]:
play_counts_df.to_feather("input_data/play_counts_df.feather")